In [23]:
import numpy as np 
import pandas as pd


from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB

from sklearn.linear_model import LogisticRegression

from sklearn import tree
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier 

In [13]:
def read_data_from_csv(file_path: str) -> pd.DataFrame:
    return pd.read_csv(file_path)

In [ ]:
REVIEWS_DATASET_PATH = './datasets/Womens_Clothing_E-Commerce_Reviews.csv'
reviews_dataset = read_data_from_csv(file_path=REVIEWS_DATASET_PATH)

## Resumen del dataset

El dataset contiene 23486 filas, cada una correspondiente a una reseña con unas 10 columnas correspondiente a variables explicadas a continuación.
### Descripción de cada columna

| Columna  | Descripción  |
|---|---|
| Clothing ID | Identificador numérico de la prenda especificada en la crítica |
| Age | Valor numérico correspondiente a la edad de la persona que realizo la crítica |
| Title | Titulo o encabezado de la crítica |
| Reviw Text | Contenido de la reseña realizada |
| Rating | Valor numérico en el rango del 1 al 5, siendo el 1 representante de máxima inconformidad y el 5 correspondiente al mejor valor posible |
| Recommended IND | Valor entre 0 y 1, representando respectivamente si el cliente no recomienda el producto o por el contrario si lo recomienda |
| Positive Feedback Count | Cantidad de reseñas positivas recibidas para el mismo producto |
| Division Name | Nombre de la división a la cual pertenece el producto |
| Department Name | Nombre del departamento al cual pertenece el producto |
| Class Name | Nombre de la clase al cual pertenece el producto |

### Categoria de variables

- Variables Cuantitativas : Age, Positive Feedback Count.

- Variables Cualitativas : Clothing ID, Title, Review Text, Rating, Recommended IND, Division Name, Department Name, Class Name.


In [ ]:
reviews_dataset.dtypes

In [ ]:
reviews_dataset

## Limpieza de Datos

### DATOS NULOS

Observamos la cantidad de nulos que poseen cada columna y el porcentaje que representan dicha cantidad de datos en cada columna

In [ ]:
reviews_dataset.isna().sum()

In [ ]:
reviews_dataset.isnull().sum() *100 / len(reviews_dataset.index)

Debido a que el trabajo a realizar con el dataset consiste en clasificar una reseña como una crítica positiva o una negativa a partir del texto en el campo “Review Text”, decidimos eliminar las columnas que no resultan de utilidad para dicho objetivo. De esta forma, obtenemos un dataset con las columnas necesarias para identificar las distintas prendas, el texto escrito en la reseña y por último la columna "Rating" para el desarrollo del ítem D, motivo por el cual conservamos dicha columna.

In [ ]:
columns_eliminate= ['Unnamed: 0','Age','Title','Recommended IND','Positive Feedback Count','Division Name','Department Name','Class Name']
reviews_dataset.drop(columns_eliminate,axis = 'columns',inplace = True)

reviews_dataset

Por ultimo dado a que las filas con valores inválidos en la columna de "Review Text" son un total de 845 que representan una cantidad menor al 4% de la totalidad de los datos, optamos por eliminar dichas filas.

In [ ]:
reviews_dataset.dropna(inplace=True)
reviews_dataset.shape

## Construccion de una nueva variable:

## Verificación:
Compruebo que la variable Rating no posea valores fuera de rango (1 al 5) antes de operar con dicha columna

In [ ]:
reviews_dataset.Rating.value_counts()

In [ ]:
reviews_dataset['Rating Category'] = reviews_dataset['Rating'].apply(lambda x: 'Negativo' if x <= 3 else 'Positivo')
reviews_dataset

In [ ]:
reviews = reviews_dataset.loc[:, ['Review Text', 'Rating Category']]
reviews

In [ ]:
le = LabelEncoder()
reviews['Rating Category'] = le.fit_transform(reviews['Rating Category'])
reviews

In [ ]:
reviews.dtypes

In [ ]:
stop_words = set(stopwords.words('english'))
vect = CountVectorizer(stop_words=stop_words)
vect_df = vect.fit_transform(reviews['Review Text'].values.tolist())
vect_df.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(vect_df, reviews['Rating Category'], test_size=0.3, random_state=10)

### Naive Bayes

In [ ]:
nb_clf = MultinomialNB()
nb_clf.fit(X_train, y_train)

In [ ]:
y_nb_pred = nb_clf.predict(X_test)
print(classification_report(y_test, y_nb_pred, digits=3))

### Regresión Logística

Para poder utilizar l función, Fit se deben verificar dos cosas:

i. Que no haya datos NaN

ii. Que cada elemento en la fila X_train, sea un vector.

In [ ]:
lr = LogisticRegression(C=0.5, max_iter=150)
lr.fit(X_train, y_train)

In [ ]:
y_lr_pred = lr.predict(X_test)
print(classification_report(y_test, y_lr_pred, digits=3))

### Árboles de decisión

Utilizamos Cross Validation para buscar los hiperparámetros del modelo que permitan obtener los mejores resultados. De esta forma, utilizaremos una de las métricas para buscar los valores que den mejores resultados y utilizarlos para crear nuestro modelo. Dada la cantidad de parámetros y el rango de los mismos utilizamos una randomSearch en lugar de una GridSearch, fijando la cantidad de iteraciones en 20.

In [ ]:
params = {'criterion':['gini','entropy'],
               'min_samples_split': list(range(2,25)),
               'min_samples_leaf':list(range(1,10)),
               'ccp_alpha':np.linspace(0,0.05,2),
               'max_depth':list(range(1,15))}

base_tree = tree.DecisionTreeClassifier() 


tree_random_cv = RandomizedSearchCV(estimator=base_tree,param_distributions = params, scoring= "f1", cv= 7, n_iter= 20, random_state = 10) 

tree_random_cv.fit(X_train,y_train)

print(tree_random_cv.best_params_)
print(tree_random_cv.best_score_)

In [ ]:
tree_model = tree_random_cv.best_estimator_

y_tree_pred = tree_model.predict(X_test)

print(classification_report(y_test, y_tree_pred, digits=3))

### Random Forest

Al igual que para un arbol de decicion buscamos los hiperparametros que nos den mejores resultados para el modelo de Random Forest. En el modelo de Random Forest vamos a utilizar cross validation para los hipervalores: 'criterion', 'min_samples_split', 'min_samples_leaf', 'ccp_alpha' y 'max_depth'.

In [ ]:
params = {'criterion':['gini','entropy'],'min_samples_split': list(range(2,25)),'min_samples_leaf':list(range(1,10)),
          'ccp_alpha':np.linspace(0,0.05,2),'max_depth':list(range(1,15)),'n_estimators': [15,30,60] }


forest_model= RandomForestClassifier() 

r_forest_random_cv = RandomizedSearchCV(estimator=forest_model,param_distributions = params, scoring= 'f1', cv= 7, 
                                      n_iter= 20, random_state = 10) 

r_forest_random_cv.fit(X_train,y_train)

print(r_forest_random_cv.best_params_)
print(r_forest_random_cv.best_score_)

## Aclaración:
Se utilizo RandomSearch para realizar el proceso de CrossValidation con el fin de utilizar un rango de valores para los hiperparametros, lo que por el contrario si se utiliza GridSearch se debe dar una cantidad de posibles valores para cada parametro mas reducida.

In [ ]:
best_forest_model = r_forest_random_cv.best_estimator_

y_forest_pred = best_forest_model.predict(X_test)

print(classification_report(y_test, y_forest_pred, digits=3))